In [15]:
import json
import requests
from datetime import datetime, timedelta
from pprint import pprint
config = json.load( open('analytics.dat') )
client_id = config['client_id']
client_secret = config['client_secret']
refresh_token = config['refresh_token']
def update_token(client_id, client_secret, refresh_token):
    """Обновление токена для запросов к API. Возвращает токен"""    
    url_token = 'https://accounts.google.com/o/oauth2/token'
    params = { 'client_id' : client_id, 'client_secret' : client_secret, 
               'refresh_token' : refresh_token, 'grant_type' : 'refresh_token' }
    r = requests.post( url_token, data = params )  
    print('Токен выдан до {}'.format(datetime.today() + timedelta( hours = 1 )))
    return r.json()['access_token']

In [16]:
token = update_token(client_id, client_secret, refresh_token)

Токен выдан до 2019-01-17 16:40:34.263045


In [17]:
url = 'https://analyticsreporting.googleapis.com/v4/reports:batchGet'
profile_id = '71639180'
start_date = '2018-01-01'
end_date = '2018-01-07'
metrics = [ { 'expression': 'ga:sessions' }, { 'expression': 'ga:pageviews' } ]
dimensions = [ { 'name': 'ga:date' } ]

In [18]:
params = {
    "reportRequests": [
        {
            'viewId': profile_id,
            'dateRanges': [ { 'startDate': start_date, 'endDate': end_date } ],
            'metrics': metrics,
            'dimensions': dimensions
        } 
    ]
}

In [19]:
headers = { 'Authorization': 'Bearer ' + token }

In [20]:
r = requests.post( url, json = params, headers = headers )

In [27]:
pprint(r.json())


{'reports': [{'columnHeader': {'dimensions': ['ga:date'],
                               'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions',
                                                                         'type': 'INTEGER'},
                                                                        {'name': 'ga:pageviews',
                                                                         'type': 'INTEGER'}]}},
              'data': {'isDataGolden': True,
                       'maximums': [{'values': ['137', '194']}],
                       'minimums': [{'values': ['71', '89']}],
                       'rowCount': 7,
                       'rows': [{'dimensions': ['20180101'],
                                 'metrics': [{'values': ['71', '89']}]},
                                {'dimensions': ['20180102'],
                                 'metrics': [{'values': ['77', '106']}]},
                                {'dimensions': ['20180103'],
                   

In [60]:
tmp = 0
for i in range(len(r.json()['reports'][0]['data']['rows'])):
    tmp += int(r.json()['reports'][0]['data']['rows'][i]['metrics'][0]['values'][0])
print(tmp)

764
